BPMN Workflow (Microservices)
----------------------------------------------

![](images/Microservices-BPMN.png)

- - -

The [Business Process Model and Notation](https://de.wikipedia.org/wiki/Business_Process_Model_and_Notation) (BPMN, German business process model and notation) is a graphical specification language in business informatics and process management. It provides symbols that technical, method and IT specialists can use to model and document business processes and workflows.

[Camunda BPM](https://camunda.com/de/) is a free workflow management system written in Java that allows business processes to be defined and executed in BPMN 2.0.

The following BPMN example is based on the blog entry [Use Camunda without touching Java and get an easy-to-use REST-based orchestration and workflow engine](https://blog.bernd-ruecker.com/use-camunda-without-touching-java-and-get-an-easy-to-use-rest-based-orchestration-and-workflow-7bdf25ac198e) by Bernd Rücker.

- - -
The example consists of the following microservices

* A BPMN **Workflow Engine** (Camunda) with an invoice approval process.
* A **frontend** implemented in HTML and JS to start the above process.
* A **backend** to post the invoices implemented in Java.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-backend.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-frontend.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/camunda.yaml

- - -
After starting the microservices we need to publish the BPMN process in the workflow engine.

In [ ]:
! wget https://raw.githubusercontent.com/mc-b/bpmn-tutorial/master/RechnungStep3.bpmn
! curl -w "\n" -H "Accept: application/json" -F "deployment-name=rechnung" -F "enable-duplicate-filtering=true" -F "deploy-changed-only=true" -F "RechnungStep3.bpmn=@RechnungStep3.bpmn" http://camunda:8080/engine-rest/deployment/create    

- - -
Now we can enter new invoices via the frontend and view them in the workflow engine. The URLs are as follows:

In [ ]:
! echo "BPMN Workflow Engine: http://"$(cat ~/work/server-ip)/camunda # User/Password: demo"
! echo "Frontend            : http://"$(cat ~/work/server-ip)/frontend/

In the `Cockpit` under `Process Definitions` a process `InvoiceStep3` should be visible. If this is missing, run the command above again to publish the process.

Using the frontend or following command we can start this process. The started process can then be seen under `Tasklist`.


In [ ]:
%%bash
curl \
-H "Content-Type: application/json" \
-X POST \
-d '{ "variables": { "rnr": {"value": "123", "type": "long"}, "rbetrag": {"value": "200.00", "type": "String"} } }' \
http://camunda:8080/engine-rest/process-definition/key/RechnungStep3/start

- - -
Clean up

In [ ]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-backend.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-frontend.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/camunda.yaml